# Setup

## Constants

In [1]:
V_CC = 15

# BJT Constants
V_T = 26e-3
V_BE = 0.7
beta = 150
V_A = 65

# FET Constants
K_N = 5e-3
V_TN = -2
LAMBDA = 0.01

# Q1 Values
R_S = 8e3
R_11 = 1e3
R_21 = 501
R_C1 = 1e3
R_E1 = 500

# Q2 Values
R_12 = 6e3
R_22 = 8e3
R_C2 = 2e3
R_E2 = 3.8e3

# M3 Values
R_13 = 1e3
R_23 = 6.5e3
R_SS = 80e3
R_L = 300

## Utility

In [2]:
from sympy import Function, Pow


def par_res(*res: list[float]) -> float:
    return sum(map(lambda x: x**-1, res))**-1


class ParallelResistor(Function):
    """A sympy function representing resistors in parallel."""
    def _latex(self, printer):
        return "(" + "||".join(map(printer._print, self.args)) + ")"

    def _eval_evalf(self, prec):
        return par_res(*self.args)._eval_evalf(prec)

## Symbols

In [3]:
from sympy import symbols
V_CCs = symbols("V_CC")

# BJT Constants
V_Ts, V_BEs, BETAs, V_As = symbols("V_T V_BE beta V_A")

# FET Constants
K_Ns, V_TNs, LAMBDAs = symbols("K_n V_TN lambda")

# Q1 Values
R_Ss, R_11s, R_21s, R_C1s, R_E1s = symbols("R_S R_11 R_21 R_C1 R_E1")

# Q2 Values
R_12s, R_22s, R_C2s, R_E2s = symbols("R_12 R_22 R_C2 R_E2")

# M3 Values
R_13s, R_23s, R_SSs, R_Ls = symbols("R_13 R_23 R_SS R_L")

# DC Analysis

## Q1

In [23]:
import sympy

I_B1s = sympy.Symbol("I_B1")
R_THs, V_THs = sympy.symbols("R_TH V_TH")

R_TH = par_res(R_11, R_22)
V_TH = V_CC * R_21 / (R_11 + R_21)

expr = sympy.solve(-V_THs + I_B1s * R_THs + V_BEs + (1 + BETAs) * I_B1s * R_E1s, I_B1s)[0]
display(expr)

I_B1 = expr.subs({ V_BEs: V_BE, V_THs: V_TH, R_E1s: R_E1, R_THs: R_TH, BETAs: beta })
I_C1 = beta * I_B1

(-V_BE + V_TH)/(R_E1*beta + R_E1 + R_TH)

## Q2

In [ ]:
R_TH = ParallelResistor(R_12, R_22)
V_TH = V_CC * R_22 / (R_12 + R_22)
expr = sympy.solve(-V_THs + I_B * R_THs + V_BEs + (1 + betas) * I_B * R_Es, I_B)[0]
display(expr)
I_B2 = expr.subs({ V_BEs: V_BE, V_THs: V_TH, R_Es: R_E2, R_THs: R_TH, betas: beta })
I_C2 = beta * I_B2

## M3

In [ ]:
V_G = V_CC * R_2s / (R_2s + R_1s)
print(f"V_G: {V_G}")
V_D = V_CC
print(f"V_D: {V_D}")

V_S = I_D * R_SS
I_D_expr = K_N * (V_G - V_S - V_TN)**2
display(I_D_expr)
I_D1, I_D2 = sympy.solve(I_D_expr - I_D, I_D)
print(f"Potential I_D: {I_D1}, {I_D2}")

V_S1 = I_D1 * R_SS
print(f"V_S1: {V_S1}")
V_S2 = I_D2 * R_SS
print(f"V_S2: {V_S2}")

V_GS1 = V_G - V_S1
V_GS2 = V_G - V_S2
V_DS1 = V_D - V_S1
V_DS2 = V_D - V_S2

I_D1_valid = V_DS1 >= (V_GS1 - V_TN) and V_GS1 > V_TN
I_D2_valid = V_DS2 >= (V_GS2 - V_TN) and V_GS2 > V_TN
print(I_D1_valid, I_D2_valid)

assert I_D1_valid or I_D2_valid, "No Valid I_D Found"
if I_D1_valid:
    I_D = I_D1
else:
    I_D = I_D2
print(f"I_D: {I_D}")

# AC Analysis

In [ ]:
import numpy as np

# Q1
r_pi1 = V_T / I_B1
g_m1 = I_C1 / V_T
r_01 = V_A / I_C1
print(f"r_pi1: {r_pi1}")
print(f"g_m1: {g_m1}")
print(f"r_01: {r_01}")

# Q2
r_pi2 = V_T / I_B2
g_m2 = I_C2 / V_T
r_02 = V_A / I_C2
print(f"r_pi2: {r_pi2}")
print(f"g_m2: {g_m2}")
print(f"r_02: {r_02}")

# M3
g_m3 = 2 * np.sqrt(K_N * float(I_D))
r_03 = 1 / (LAMBDA * I_D)
print(f"g_m3: {g_m3}")
print(f"r_03: {r_03}")

In [ ]:
import sympy

# Q1 Symbols
v_pi1 = sympy.symbols("v_pi_1")
r_pi1s, g_m1s, r_01s = sympy.symbols("r_pi_1 g_m_1 r_0_1")
R_11s, R_21s, R_C1s, R_Si = sympy.symbols("R_1_1 R_2_1 R_C_1 R_S")

# Q2 Symbols
v_pi2 = sympy.symbols("v_pi_2")
r_pi2s, g_m2s, r_02s = sympy.symbols("r_pi_2 g_m2 r_02")
R_12s, R_22s, R_C2s = sympy.symbols("R_12 R_22 R_C2")

# M3 Symbols
V_g, V_gs = sympy.symbols("V_g V_gs")
g_m3s, r_03s = sympy.symbols("g_m3 r_03")
R_13s, R_23s, R_SSs, R_Ls = sympy.symbols("R_13 R_23 R_SS R_L")

# Branch 1
R_1_p_R_2 = ParallelResistor(R_11s, R_21s, r_pi1s)
V_i = v_pi1 * (R_1_p_R_2 + R_Si) / R_1_p_R_2
# display(V_i)

# Branch 2
eq1 = -g_m1s * v_pi1 * ParallelResistor(r_01s, R_12s, R_22s, R_C1s, r_pi2s) - v_pi2
# display(eq1)

# Branch 3
eq2 = -g_m2s * v_pi2 * ParallelResistor(r_02s, R_C2s, R_13s, R_23s) - V_g
# display(eq2)

# Branch 4
V_o = g_m3s * V_gs * ParallelResistor(r_03s, R_Ls, R_SSs)
# display(V_o)
eq3 = V_g - V_o - V_gs
# display(eq3)

# Gain
A_v = (V_o / V_i).subs({
    v_pi1: sympy.solve(eq1, v_pi1)[0],
    v_pi2: sympy.solve(eq2, v_pi2)[0],
    V_gs: sympy.solve(eq3, V_gs)[0]
})
display(A_v)
A_v.subs({ 
    g_m1s: g_m1, g_m2s: g_m2, g_m3s: g_m3,
    r_01s: r_01, r_02s: r_02, r_03s: r_03,
    r_pi1s: r_pi1, r_pi2s: r_pi2,
    R_Si: R_S, R_11s: R_11, R_21s: R_21, R_C1s: R_C1,
    R_12s: R_12, R_22s: R_22, R_C2s: R_C2,
    R_SSs: R_SS, R_Ls: R_L, R_13s: R_13, R_23s: R_23,
}).evalf()